## Item Based Collaborative Filtering

In [1]:
import numpy as np
import pandas as pd

In [2]:
r_cols = ['userId', 'movieId', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols,
                      encoding='latin-1')
m_cols = ['movieId', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

In [3]:
def replace_name(x):
    return movies[movies['movieId']==x].title.values[0]
ratings.movieId = ratings.movieId.map(replace_name)

In [4]:
item_ratings = ratings.pivot_table(index = ['movieId'], columns = ['userId'], values = 'rating', fill_value=0.0)

In [5]:
random_row = np.random.choice(item_ratings.index,1)
random_col = np.random.choice(item_ratings.columns,1)

In [6]:
test_item = item_ratings.loc[random_row[0]].tolist()
test_item = pd.Series(test_item, index=item_ratings.columns)
test_user = item_ratings[random_col[0]].tolist()
test_user = pd.Series(test_user, index=item_ratings.index)

In [11]:
#predict ratings on test_user's test item based on it's correlation with other items test_user has rated
def find_recommendation(neighbours):
    sm = 0
    cnt = 0
    for i,y in enumerate(test_user):
        for j in neighbours:
            if test_user.index[i] == j[0] and y>0.0:
                sm += y*j[1]
                cnt += j[1]
    if cnt!=0:
        print("{} predicted rating for user {} : {}".format(random_row[0],random_col[0],round(sm/cnt,1)))

In [12]:
#selecting top similar items based on pearson correlation
test_item_similarity = item_ratings.corrwith(test_item, axis = 1)
neighbours = []
for i,y in enumerate(test_item_similarity):
    if y>0 and y!=1:
        neighbours.append((test_item_similarity.index[i],y))
find_recommendation(neighbours)

Top Gun (1986) predicted rating for user 871 : 3.8
